### Import Libraries

In [1]:
!pip install torch_geometric
!pip install networkit
!apt install libgraphviz-dev
!pip install pygraphviz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 50.9 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libgvc6-plugins-gtk
  librsvg2-common libxdot4
Suggested packages:
  gvfs
The following NEW packages will be installed:
  libgail-common libgail18 libgraphviz-dev libgtk2.0-0 libgtk2.0-bin libgtk2.0-common
  libgvc6-plugins-gtk librsvg2-common libxdot4
0 upgraded, 9 newly installed, 0 to remove and 45 not upgraded.
Need to get 2,433 kB of archives.
After this operation, 7,694 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libgtk2.0-common all 2.24.33-2ubuntu2 [125 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libgtk2.0-0 amd64 2.24.33-2ubuntu2 [2,037

In [26]:
import numpy as np
import pandas as pd
import torch
from torch import nn

import json
import os
import networkx as nx
import networkit as nk
from networkit import vizbridges

from tqdm import tqdm
import ast
import matplotlib.pyplot as plt
import re
from sklearn.metrics import top_k_accuracy_score
from sklearn.metrics import ndcg_score

from torch_geometric.data import Data
from torch_geometric.nn import GATv2Conv

import copy
from typing import Callable, Tuple

import torch
from torch import Tensor
from torch.nn import Module, Parameter

from torch_geometric.nn.inits import reset, uniform
import random

In [3]:
torch.manual_seed(42)
np.random.seed(42)

### Util functions

In [4]:
def load_dataset_timestamp( n_users, n_context, seq_len):
    act_list = []
    time_list = []
    user_list = []

    max_timestamp = -1.0
    min_timestamp = float('inf')

    with open('gowalla_user_activity.txt', 'r') as raw_file:
        for line in raw_file:
            t_item_list = []
            t_time_list = []
            user = int(line.split(':')[0])
            entries = line.split()[1:]
            for a_entry in entries:
                item, time_stamp = a_entry.split(':')
                t_item_list.append(int(item.strip()))
                t_time_list.append(int(time_stamp.strip()))

                if min_timestamp > int(time_stamp.strip()):
                    min_timestamp = int(time_stamp.strip())
                if max_timestamp < int(time_stamp.strip()):
                    max_timestamp = int(time_stamp.strip())

            act_list.append(t_item_list[0: seq_len])
            time_list.append(t_time_list[0: seq_len])
            user_list.append(user)

    new_time_list = []
    num_bins = 0

    times_bins = np.linspace(min_timestamp, max_timestamp + 1, num=num_bins, dtype=np.int32)
    for a_time_list in time_list:
        temp_time_list = (np.digitize(np.asarray(a_time_list), times_bins) - 1).tolist()
        new_time_list.append(temp_time_list)

    all_examples = {}
    for i in range(0, len(act_list)):

        train_act_seq = act_list[i][:-2]

        train_time_seq = new_time_list[i][:-2]

        train_act_label = act_list[i][-2]
        train_time_label = new_time_list[i][-2]

        test_act_seq = act_list[i][1:-1]
        test_time_seq = new_time_list[i][1:-1]

        test_act_label = act_list[i][-1]
        test_time_label = new_time_list[i][-1]

        entry = {
            'train_act_seq': train_act_seq,
            'train_time_seq': train_time_seq,
            'train_act_label': train_act_label,
            'train_time_label': train_time_label,
            'test_act_seq': test_act_seq,
            'test_time_seq': test_time_seq,
            'test_act_label': test_act_label,
            'test_time_label': test_time_label,
            'seq_len': len(train_act_seq),
            'user': user_list[i]
        }

        all_examples[user_list[i]] = entry

    return all_examples

### Metrics

In [5]:
def print_metrics(hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100):
    print(f'hits@1: {hits1:.6f}, hits@5: {hits5:.6f}, hits@10: {hits10:.6f}, hits@20: {hits20:.6f}')
    print(f'hits@50: {hits50:.6f}, hits@100: {hits100:.6f}')
    print(f'map@1: {map1:.6f}, map@5: {map5:.6f}, map@10: {map10:.6f}, map@20: {map20:.6f}')
    print(f'map@50: {map50:.6f}, map@100: {map100:.6f}')
    print(f'ndcg@1: {ndcg1:.6f}, ndcg@5: {ndcg5:.6f}, ndcg@10: {ndcg10:.6f}, ndcg@20: {ndcg20:.6f}')
    print(f'ndcg@50: {ndcg50:.6f}, ndcg@100: {ndcg100:.6f}')

In [6]:
def apk(actual, predicted, k=10):

    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)


def mapk(y_prob, y, k=10):

    predicted = [np.argsort(p_)[-k:][::-1] for p_ in y_prob]
    actual = [[y_] for y_ in y]
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])


def hits_k(y_prob, y, k=10):
    acc = []
    for p_, y_ in zip(y_prob, y):
        top_k = p_.argsort()[-k:][::-1]
        acc += [1. if y_ in top_k else 0.]
    return sum(acc) / len(acc)

In [7]:
def get_metrics_(probs, labels_batch, test_one_hot):
    hits1 = top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=1, labels = classes)
    hits5 = top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=5, labels = classes)
    hits10 = top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=10, labels = classes)
    hits20 = top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=20, labels = classes)
    hits50= top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=50, labels = classes)
    hits100 = top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=100, labels = classes)

    map1 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=1)
    map5 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=5)
    map10 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=10)
    map20 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=20)
    map50 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=50)
    map100 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=100)

    ndcg1 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=1)
    ndcg5 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=5)
    ndcg10 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=10)
    ndcg20 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=20)
    ndcg50 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=50)
    ndcg100 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=100)
    return hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100

### Graph construction

In [8]:
users = load_dataset_timestamp(20001, 128, 30)

In [9]:
friends = pd.read_csv('gowalla_edges.csv')

In [ ]:
Gcl = nx.Graph()
for i in range(len(users)):
  if users[i]['user'] not in Gcl.nodes():
      Gcl.add_node(users[i]['user'], weight=10, color='seagreen')
for i in range(len(friends)):
  Gcl.add_edge(friends.loc[i, '1st friend'], friends.loc[i, '2nd friend'], color = 'blue')



nodes = list(Gcl.nodes())
node_colors = [Gcl.nodes[node]['color'] for node in Gcl.nodes]
node_weights = [Gcl.nodes[node]['weight'] for node in Gcl.nodes]
edge_colors = [Gcl.edges[edge]['color'] for edge in Gcl.edges]

plt.figure(figsize=(12, 8))
pos = nx.drawing.nx_agraph.graphviz_layout(Gcl, prog='neato')
nx.draw_networkx(Gcl, pos=pos, with_labels=False, node_size=node_weights, node_color=node_colors, edge_color=edge_colors, width=1)

In [10]:
Gcl = nx.Graph()
for i in range(len(users)):
  if users[i]['user'] not in Gcl.nodes():
      Gcl.add_node(users[i]['user'], weight=10, color='seagreen')
for i in range(len(friends)):
  Gcl.add_edge(friends.loc[i, '1st friend'], friends.loc[i, '2nd friend'], color = 'blue')


nodes = list(Gcl.nodes())
node_colors = [Gcl.nodes[node]['color'] for node in Gcl.nodes]
node_weights = [Gcl.nodes[node]['weight'] for node in Gcl.nodes]
edge_colors = [Gcl.edges[edge]['color'] for edge in Gcl.edges]

In [11]:
graphid2pid = {}
pid2graphid = {}

s = 0
for node_id in Gcl.nodes():
    pid2graphid[node_id] = s
    graphid2pid[s] = node_id
    s += 1

graph_edges = []
for (u, v) in Gcl.edges():
    graph_edges.append((pid2graphid[u], pid2graphid[v]))

### Graph characteristics

In [12]:
nkg = nk.Graph(n=len(pid2graphid), weighted=True, directed=False, edgesIndexed=True)
for (u, v) in graph_edges:
    nkg.addEdge(u, v)

In [13]:
nk.overview(nkg)

Network Properties:
nodes, edges			20001, 167752
directed?			False
weighted?			True
isolated nodes			275
self-loops			0
density				0.000839
clustering coefficient		0.294269
min/max/avg degree		0, 6171, 16.774361
degree assortativity		-0.064750
number of connected components	290
size of largest component	19698 (98.49 %)


### User dataset construction

In [14]:
class UserInfoDataset():
  def __init__(self, data, max_len):
    self.data = data
    self.max_len = max_len

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):

    user = self.data[idx]
    seq_len = user['seq_len']

    tr_act_seq = np.zeros((self.max_len,)).astype('int32')
    tr_act_seq[:seq_len] = np.array(user['train_act_seq'])
    tr_act_seq = np.transpose(tr_act_seq)

    tr_time_seq = np.zeros((self.max_len,)).astype('int32')
    tr_time_seq[:seq_len] = user['train_time_seq']
    tr_time_seq = np.transpose(tr_time_seq)

    t_act_seq = np.zeros((self.max_len, )).astype('int32')
    t_act_seq[:seq_len] = user['test_act_seq']
    t_act_seq = np.transpose(t_act_seq)

    t_time_seq = np.zeros((self.max_len, )).astype('int32')
    t_time_seq[:seq_len] = user['test_time_seq']
    t_time_seq = np.transpose(t_time_seq)


    return user['user'], tr_act_seq, \
    tr_time_seq, user['train_act_label'], \
    user['train_time_label'], t_act_seq, \
    t_time_seq, user['test_act_label'], \
    user['test_time_label'], user['seq_len']

In [15]:
from torch.utils.data import Subset
from torch.utils.data import DataLoader

user_dataset = UserInfoDataset(users, 30)

n = len(user_dataset)

indices = np.arange(n)
indices = np.random.permutation(indices)

train_indices = indices [:int(0.8*n)]
test_indices = indices[int(0.8*n):]

user_train_dataset = Subset(user_dataset, train_indices)
user_test_dataset = Subset(user_dataset, test_indices)

user_dataloader = DataLoader(user_dataset, batch_size = 64, shuffle = True)
user_train_dataloader = DataLoader(user_train_dataset, batch_size=64, shuffle=True)
user_test_dataloader = DataLoader(user_test_dataset, batch_size=64, shuffle=True)

item_emb  = nn.init.xavier_uniform_(torch.empty(186, 128))

## Pre-training

In [16]:
rnn = nn.RNN(128, 128, batch_first = True)
norm = nn.BatchNorm1d(128)

### Getting initial user representations

In [17]:
user_representations = {}
for users, train_input, train_time, train_label, train_time_label, test_input, test_time, test_label, test_time_label, seq_len in user_dataloader:
      comb_input = np.concatenate([np.expand_dims(train_input, axis=-1),
                                                np.expand_dims(train_time, axis=-1)], axis=2)
      model_input = comb_input
      model_output = train_label
      rnn_input_emb = item_emb[model_input[:, :, 0]]
      x, h = rnn(rnn_input_emb)
      hx = torch.zeros(x.shape[0], x.shape[2])#.to(device)
      for i in range(hx.shape[0]):
        hx[i] = x[i][seq_len[i] - 1]
      user_representation = norm(hx)
      for i in range(len(users)):
        user_representations[users[i].item()] = (user_representation[i], train_label[i].item(), seq_len[i].item())

In [18]:
features = []
seq_lens = []
for graphid in Gcl.nodes():
    pid = graphid2pid[graphid]
    features.append(user_representations[graphid][0].detach().numpy())
    seq_lens.append(user_representations[graphid][2])
features = np.array(features)

In [19]:
dataset = Data(x=torch.tensor(features, dtype=torch.float), edge_index=torch.tensor(np.array(graph_edges).T, dtype=torch.int64))

### Deep Graph Infomax model

In [20]:
eps = 1e-15
class DeepGraphInfomax(torch.nn.Module):
    def __init__(
        self,
        hidden_channels,
        encoder,
        summary,
        corruption,
    ):
        super().__init__()
        self.hidden_channels = hidden_channels
        self.encoder = encoder
        self.summary = summary
        self.corruption = corruption

        self.weight = Parameter(torch.empty(hidden_channels, hidden_channels))

        self.reset_parameters()

    def reset_parameters(self):
        reset(self.encoder)
        reset(self.summary)
        uniform(self.hidden_channels, self.weight)

    def forward(self, *args, **kwargs) -> Tuple[Tensor, Tensor, Tensor]:
        pos_z = self.encoder(*args, **kwargs)

        cor = self.corruption(*args, **kwargs)
        cor = cor if isinstance(cor, tuple) else (cor, )
        cor_args = cor[:len(args)]
        cor_kwargs = copy.copy(kwargs)
        for key, value in zip(kwargs.keys(), cor[len(args):]):
            cor_kwargs[key] = value

        neg_z = self.encoder(*cor_args, **cor_kwargs)

        summary = self.summary(pos_z, *args, **kwargs)

        return pos_z, neg_z, summary

    def discriminate(self, z: Tensor, summary: Tensor,
                     sigmoid: bool = True) -> Tensor:

        summary = summary.t() if summary.dim() > 1 else summary
        value = torch.matmul(z, torch.matmul(self.weight, summary))
        return torch.sigmoid(value) if sigmoid else value

    def loss(self, pos_z: Tensor, neg_z: Tensor, summary: Tensor) -> Tensor:
        pos_loss = -torch.log(
            self.discriminate(pos_z, summary, sigmoid=True) + eps).mean()
        neg_loss = -torch.log(1 -
                              self.discriminate(neg_z, summary, sigmoid=True) +
                              eps).mean()

        return pos_loss + neg_loss


    def __repr__(self) -> str:
        return f'{self.__class__.__name__}({self.hidden_channels})'

In [21]:
class Encoder(nn.Module):
    def __init__(self, interm_channels, hidden_channels, num_features):
        super().__init__()

        self.num_features = num_features
        self.rnn = nn.RNN(128, 128, batch_first = True)
        self.norm = nn.BatchNorm1d(128)
        self.conv1 = GATv2Conv(self.num_features, interm_channels)
        self.prelu1 = nn.PReLU(interm_channels)
        self.dropout = nn.Dropout(0.2)
        self.conv2 = GATv2Conv(interm_channels, hidden_channels)
        self.prelu2 = nn.PReLU(hidden_channels)
        self.conv3 = GATv2Conv(hidden_channels, hidden_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.prelu1(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = self.prelu2(x)
        x = self.dropout(x)
        x = self.conv3(x, edge_index)
        return x

### Augmentation construction helping functions

In [22]:
def index_to_mask(index):

    index = index.view(-1)
    size = int(index.max()) + 1

    mask = index.new_zeros(20001, dtype=torch.bool)
    mask[index] = True
    return mask

In [23]:
def map_index(indices, edge_index):
  dictionary = {}
  label = 0
  for ind in indices:
    dictionary[ind] = label
    label += 1
  for i in range(edge_index.shape[1]):
    edge_index[0][i] = dictionary[edge_index[0][i].item()]
    edge_index[1][i] = dictionary[edge_index[1][i].item()]
  return edge_index

In [24]:
def construct_neighbors_dictionary(edge_index):
  dictionary = {}
  for i in range(edge_index.shape[1]):
    if edge_index[0][i].item() not in dictionary.keys():
      dictionary[edge_index[0][i].item()] = []
    dictionary[edge_index[0][i].item()].append(edge_index[1][i].item())
    if edge_index[1][i].item() not in dictionary.keys():
      dictionary[edge_index[1][i].item()] = []
    dictionary[edge_index[1][i].item()].append(edge_index[0][i].item())
  return dictionary

In [25]:
neighbors = construct_neighbors_dictionary(dataset.edge_index)

### Augmentations

In [27]:
def node_mix(x, edge_index):
    return x[torch.randperm(x.size(0))], edge_index

In [28]:
def node_dropout(x, edge_index):
  indices = []
  deleted_indices = []
  for i in range(x.shape[0]):
    drop = random.random()
    if drop >= 0.3:
      indices.append(i)
    else:
      deleted_indices.append(i)
  x = torch.index_select(x, 0, torch.tensor(indices, dtype = torch.int32))
  node_mask = index_to_mask(torch.tensor(indices, dtype = torch.int32))
  edge_mask = node_mask[edge_index[0]] & node_mask[edge_index[1]]
  edge_index = edge_index[:, edge_mask]
  edge_index = map_index(indices, edge_index)

  return x, edge_index

In [29]:
def random_walk(x, edge_index):
  node = 0
  rwalk = [node]
  walk_length = 10000
  for i in range(walk_length - 1):
    temp = neighbors[node]
    temp = list(set(temp) - set(rwalk))
    if len(temp) == 0:
      break
    new_node = random.choice(temp)
    rwalk.append(new_node)
    node = new_node
  x = torch.index_select(x, 0, torch.tensor(rwalk, dtype = torch.int32))
  node_mask = index_to_mask(torch.tensor(rwalk, dtype = torch.int32))
  edge_mask = node_mask[edge_index[0]] & node_mask[edge_index[1]]
  edge_index = edge_index[:, edge_mask]
  edge_index = map_index(rwalk, edge_index)

  return x, edge_index

In [30]:
def edge_dropout(x, edge_index):
  indices = []
  for i in range(edge_index.shape[1]):
    drop = random.random()
    if drop >= 0.3:
      indices.append(i)
  edge_index = torch.index_select(edge_index, 1, torch.tensor(indices, dtype = torch.int32))
  return x, edge_index

### Link prediction head

In [31]:
class LinkPredHead(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc_1 = nn.Linear(256, 1)

    def forward(self, x, edge_index):
        x_src, x_dst = x[edge_index[0]], x[edge_index[1]]
        x = torch.cat((x_src, x_dst), dim = 1)
        x = self.fc_1(x)
        return x#torch.sum(x_src * x_dst, dim=1)

### Next action prediction head

In [32]:
class NextActionPredHead(nn.Module):
  def __init__(self):
    super(NextActionPredHead, self).__init__()
    self.fc1 = nn.Linear(128, 186)
    self.rnn = nn.RNN(128, 128, batch_first = True)

  def forward(self, x):
    x = self.fc1(x)
    return x

In [33]:
def negative_sampling(edge_index, num_nodes):
    mask_1 = torch.rand(edge_index.size(1)) < 0.5
    mask_2 = ~mask_1

    neg_edge_index = edge_index.clone()
    neg_edge_index[0, mask_1] = torch.randint(num_nodes, (mask_1.sum(),))
    neg_edge_index[1, mask_2] = torch.randint(num_nodes, (mask_2.sum(),))

    return neg_edge_index

### User representations dataset

In [34]:
class UserDataset():
    def __init__(self, user_representations):
        self.user_representations = user_representations

    def __len__(self):
        return len(self.user_representations)

    def __getitem__(self, idx):

        return self.user_representations[idx][0], self.user_representations[idx][1], self.user_representations[idx][2]

### Training functions

In [37]:
def train_only_infomax(epoch):
    model.train()
    optimizer.zero_grad()
    pos_z, neg_z, summary = model(dataset.x, dataset.edge_index)

    loss_infomax = model.loss(pos_z, neg_z, summary)

    pos_out = linkmodel(pos_z, dataset.edge_index)
    neg_edge_index = negative_sampling(dataset.edge_index, dataset.num_nodes)
    neg_out = linkmodel(pos_z, neg_edge_index)

    out = torch.cat([pos_out, neg_out])
    gt = torch.cat([torch.ones_like(pos_out), torch.zeros_like(neg_out)])
    loss_link_pred = loss_bce(out, gt)

    loss =  loss_infomax
    loss.backward()
    optimizer.step()
    print(f'Epoch: {epoch}, Loss: {loss}')
    if epoch == num_epoch:
      return pos_z, neg_z, summary, loss.item()
    else:
      return loss.item()

In [38]:
def train_infomax_linkpred(epoch):
    model.train()
    optimizer.zero_grad()
    pos_z, neg_z, summary = model(dataset.x, dataset.edge_index)

    loss_infomax = model.loss(pos_z, neg_z, summary)

    pos_out = linkmodel(pos_z, dataset.edge_index)
    neg_edge_index = negative_sampling(dataset.edge_index, dataset.num_nodes)
    neg_out = linkmodel(pos_z, neg_edge_index)

    out = torch.cat([pos_out, neg_out])
    gt = torch.cat([torch.ones_like(pos_out), torch.zeros_like(neg_out)])
    loss_link_pred = loss_bce(out, gt)
    loss =  2*loss_infomax + 0.5*loss_link_pred
    loss.backward()
    optimizer.step()
    print(f'Epoch: {epoch}, Loss: {loss}')
    if epoch == num_epoch:
      return pos_z, neg_z, summary, loss.item()
    else:
      return loss.item()

In [53]:
def test(head_model):
  metrics_val = []
  head_model.eval()
  index = 0
  for users, vectors, labels in user_test_dataloader:
      test_probs = head_model(vectors)
      test_pred = torch.argmax(test_probs, axis = 1)#torch.Tensor([0]*20001)

      test_one_hot = torch.zeros(len(test_probs), num_classes)
      test_one_hot[torch.arange(len(test_one_hot)), labels] = 1


      hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100 \
       = get_metrics_(test_probs, labels, test_one_hot)

      metrics_val.append([hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100])#, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100])

  mean = torch.Tensor(metrics_val).mean(axis=0)
  test_hits1, test_hits5, test_hits10, test_hits20, test_hits50, test_hits100, test_map1, test_map5, test_map10, test_map20, test_map50, test_map100, \
  test_ndcg1, test_ndcg5, test_ndcg10, test_ndcg20, test_ndcg50, test_ndcg100 = mean

  return test_hits1, test_hits5, test_hits10, test_hits20, test_hits50, test_hits100, test_map1, test_map5, test_map10, test_map20, test_map50, test_map100, \
  test_ndcg1, test_ndcg5, test_ndcg10, test_ndcg20, test_ndcg50, test_ndcg100

In [50]:
def train2task():
  loss_fn = nn.CrossEntropyLoss()
  head_model  = NextActionPredHead()

  head_model.train()
  optimizer = torch.optim.Adam(head_model.parameters(), lr = 0.001)
  lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.97, last_epoch=-1)
  losses = []

  for i in range(200):
    for users, vectors, labels in user_train_dataloader:

        optimizer.zero_grad()

        probs = head_model(vectors)
        pred = torch.argmax(probs, axis = 1)
        one_hot = torch.zeros(len(probs), num_classes)
        one_hot[torch.arange(len(one_hot)), labels] = 1
        loss = loss_fn(probs, one_hot)

        losses.append(loss)
        loss.backward()
        optimizer.step()
    mean_loss = torch.Tensor(losses).mean(axis=0).item()
    print(f'Epoch: {i}, Loss: {mean_loss}')
  hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100 = test(head_model)
  print_metrics(hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100)
  return head_model

### Node mix augmentation, only infomax

In [42]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model = DeepGraphInfomax(hidden_channels=128,
                         encoder=Encoder(interm_channels=74, hidden_channels=128,
                                         num_features=128),
                         summary=lambda z, *args, **kwargs: torch.sigmoid(z.mean(dim=0)),
                         corruption=node_mix).to(device)
linkmodel = LinkPredHead()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005)
loss_bce = nn.BCEWithLogitsLoss()

In [54]:
num_epoch = 100
loss_epoch = []
for epoch in tqdm(range(1, num_epoch+1)):
  if epoch == num_epoch:
    pos_z, neg_z, summary, loss = train_only_infomax(epoch)
  else:
    loss = train_only_infomax(epoch)
  loss_epoch.append(loss)

  1%|          | 1/100 [00:09<15:58,  9.68s/it]

Epoch: 1, Loss: 0.3002675175666809


  2%|▏         | 2/100 [00:16<13:11,  8.07s/it]

Epoch: 2, Loss: 0.30694717168807983


  3%|▎         | 3/100 [00:22<11:45,  7.27s/it]

...

Epoch: 98, Loss: 0.10686999559402466


 99%|█████████▉| 99/100 [10:43<00:06,  6.23s/it]

Epoch: 99, Loss: 0.1630438268184662


100%|██████████| 100/100 [10:50<00:00,  6.50s/it]

Epoch: 100, Loss: 0.13529092073440552


In [55]:
final_user_representations = {}
repr = pos_z
for i in range(pos_z.shape[0]):
  final_user_representations[i] = (i, repr[i].detach(), user_representations[i][1])

In [56]:
user_dataset = UserDataset(final_user_representations)

n = len(user_dataset)

indices = np.arange(n)
indices = np.random.permutation(indices)

train_indices = indices [:int(0.8*n)]
test_indices = indices[int(0.8*n):]

user_train_dataset = Subset(user_dataset, train_indices)
user_test_dataset = Subset(user_dataset, test_indices)

user_train_dataloader = DataLoader(user_train_dataset, batch_size=64, shuffle=True)
user_test_dataloader = DataLoader(user_test_dataset, batch_size=64, shuffle=True)

In [57]:
num_classes = 186
classes = np.arange(0, num_classes)

In [58]:
head_model = train2task()
torch.save(head_model.state_dict(), 'node_mix_only_infomax_model.pth')

Epoch: 0, Loss: 4.285384654998779
Epoch: 1, Loss: 4.190764904022217
Epoch: 2, Loss: 4.137697696685791
Epoch: 3, Loss: 4.099329948425293
Epoch: 4, Loss: 4.069395542144775
Epoch: 5, Loss: 4.044259071350098
Epoch: 6, Loss: 4.022629261016846
Epoch: 7, Loss: 4.003567695617676
Epoch: 8, Loss: 3.9867825508117676
Epoch: 9, Loss: 3.971529960632324
Epoch: 10, Loss: 3.957684278488159
Epoch: 11, Loss: 3.9449241161346436
Epoch: 12, Loss: 3.933150291442871
Epoch: 13, Loss: 3.9221835136413574
Epoch: 14, Loss: 3.911931276321411
Epoch: 15, Loss: 3.902322292327881
Epoch: 16, Loss: 3.8933143615722656
Epoch: 17, Loss: 3.8847427368164062
Epoch: 18, Loss: 3.8766443729400635
Epoch: 19, Loss: 3.868969440460205
Epoch: 20, Loss: 3.861726999282837
Epoch: 21, Loss: 3.8548171520233154
Epoch: 22, Loss: 3.848130464553833
Epoch: 23, Loss: 3.841722249984741
Epoch: 24, Loss: 3.8356709480285645
Epoch: 25, Loss: 3.829836130142212
Epoch: 26, Loss: 3.8241353034973145
Epoch: 27, Loss: 3.818697214126587
Epoch: 28, Loss: 3.81

### Node dropout augmentation, only infomax

In [59]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model = DeepGraphInfomax(hidden_channels=128,
                         encoder=Encoder(interm_channels=74, hidden_channels=128,
                                         num_features=128),
                         summary=lambda z, *args, **kwargs: torch.sigmoid(z.mean(dim=0)),
                         corruption=node_dropout).to(device)
linkmodel = LinkPredHead()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005)
loss_bce = nn.BCEWithLogitsLoss()

In [60]:
num_epoch = 100
loss_epoch = []
for epoch in tqdm(range(1, num_epoch+1)):
  if epoch == num_epoch:
    pos_z, neg_z, summary, loss = train_only_infomax(epoch)
  else:
    loss = train_only_infomax(epoch)
  loss_epoch.append(loss)

  1%|          | 1/100 [00:08<13:26,  8.15s/it]

Epoch: 1, Loss: 1.4168205261230469


  2%|▏         | 2/100 [00:15<13:00,  7.96s/it]

Epoch: 2, Loss: 1.591853141784668


  3%|▎         | 3/100 [00:23<12:13,  7.56s/it]

...

Epoch: 98, Loss: 1.1193156242370605


 99%|█████████▉| 99/100 [12:46<00:07,  7.72s/it]

Epoch: 99, Loss: 0.758968710899353


100%|██████████| 100/100 [12:54<00:00,  7.74s/it]

Epoch: 100, Loss: 0.7786322832107544


In [61]:
final_user_representations = {}
repr = pos_z
for i in range(pos_z.shape[0]):
  final_user_representations[i] = (i, repr[i].detach(), user_representations[i][1])

In [62]:
user_dataset = UserDataset(final_user_representations)

n = len(user_dataset)

indices = np.arange(n)
indices = np.random.permutation(indices)

train_indices = indices [:int(0.8*n)]
test_indices = indices[int(0.8*n):]

user_train_dataset = Subset(user_dataset, train_indices)
user_test_dataset = Subset(user_dataset, test_indices)

user_train_dataloader = DataLoader(user_train_dataset, batch_size=64, shuffle=True)
user_test_dataloader = DataLoader(user_test_dataset, batch_size=64, shuffle=True)

In [63]:
num_classes = 186
classes = np.arange(0, num_classes)

In [64]:
head_model = train2task()
torch.save(head_model.state_dict(), 'node_dropout_only_infomax_model.pth')

Epoch: 0, Loss: 4.443563461303711
Epoch: 1, Loss: 4.228217601776123
Epoch: 2, Loss: 4.109987735748291
Epoch: 3, Loss: 4.02674674987793
Epoch: 4, Loss: 3.9619386196136475
Epoch: 5, Loss: 3.9084553718566895
Epoch: 6, Loss: 3.8629555702209473
Epoch: 7, Loss: 3.8232662677764893
Epoch: 8, Loss: 3.7881197929382324
Epoch: 9, Loss: 3.756572961807251
Epoch: 10, Loss: 3.7279157638549805
Epoch: 11, Loss: 3.7017226219177246
Epoch: 12, Loss: 3.677577257156372
Epoch: 13, Loss: 3.6552114486694336
Epoch: 14, Loss: 3.634401798248291
Epoch: 15, Loss: 3.6149425506591797
Epoch: 16, Loss: 3.596691131591797
Epoch: 17, Loss: 3.579517364501953
Epoch: 18, Loss: 3.563297748565674
Epoch: 19, Loss: 3.5479421615600586
Epoch: 20, Loss: 3.5333645343780518
Epoch: 21, Loss: 3.5194921493530273
Epoch: 22, Loss: 3.506268262863159
Epoch: 23, Loss: 3.4936490058898926
Epoch: 24, Loss: 3.4815561771392822
Epoch: 25, Loss: 3.4699771404266357
Epoch: 26, Loss: 3.458878517150879
Epoch: 27, Loss: 3.4482204914093018
Epoch: 28, Loss

### Edge dropout, only infomax

In [65]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model = DeepGraphInfomax(hidden_channels=128,
                         encoder=Encoder(interm_channels=74, hidden_channels=128,
                                         num_features=128),
                         summary=lambda z, *args, **kwargs: torch.sigmoid(z.mean(dim=0)),
                         corruption=edge_dropout).to(device)
linkmodel = LinkPredHead()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005)
loss_bce = nn.BCEWithLogitsLoss()

In [66]:
num_epoch = 100
loss_epoch = []
for epoch in tqdm(range(1, num_epoch+1)):
  if epoch == num_epoch:
    pos_z, neg_z, summary, loss = train_only_infomax(epoch)
  else:
    loss = train_only_infomax(epoch)
  loss_epoch.append(loss)

  1%|          | 1/100 [00:05<09:25,  5.71s/it]

Epoch: 1, Loss: 1.5530227422714233


  2%|▏         | 2/100 [00:12<09:55,  6.08s/it]

Epoch: 2, Loss: 1.5028414726257324


  3%|▎         | 3/100 [00:17<09:24,  5.82s/it]

...

Epoch: 98, Loss: 0.45930391550064087


 99%|█████████▉| 99/100 [09:48<00:06,  6.10s/it]

Epoch: 99, Loss: 0.42324739694595337


100%|██████████| 100/100 [09:53<00:00,  5.94s/it]

Epoch: 100, Loss: 0.5164247155189514


In [67]:
final_user_representations = {}
repr = pos_z
for i in range(pos_z.shape[0]):
  final_user_representations[i] = (i, repr[i].detach(), user_representations[i][1])

In [68]:
user_dataset = UserDataset(final_user_representations)

n = len(user_dataset)

indices = np.arange(n)
indices = np.random.permutation(indices)

train_indices = indices [:int(0.8*n)]
test_indices = indices[int(0.8*n):]

user_train_dataset = Subset(user_dataset, train_indices)
user_test_dataset = Subset(user_dataset, test_indices)

user_train_dataloader = DataLoader(user_train_dataset, batch_size=64, shuffle=True)
user_test_dataloader = DataLoader(user_test_dataset, batch_size=64, shuffle=True)

In [69]:
num_classes = 186
classes = np.arange(0, num_classes)

In [70]:
head_model = train2task()
torch.save(head_model.state_dict(), 'edge_dropout_only_infomax_model.pth')

Epoch: 0, Loss: 4.3176398277282715
Epoch: 1, Loss: 4.181057929992676
Epoch: 2, Loss: 4.10624361038208
Epoch: 3, Loss: 4.054159641265869
Epoch: 4, Loss: 4.013622760772705
Epoch: 5, Loss: 3.980647563934326
Epoch: 6, Loss: 3.952723979949951
Epoch: 7, Loss: 3.928549289703369
Epoch: 8, Loss: 3.907130718231201
Epoch: 9, Loss: 3.887968063354492
Epoch: 10, Loss: 3.870573043823242
Epoch: 11, Loss: 3.8547472953796387
Epoch: 12, Loss: 3.8401265144348145
Epoch: 13, Loss: 3.8266243934631348
Epoch: 14, Loss: 3.814082622528076
Epoch: 15, Loss: 3.8023440837860107
Epoch: 16, Loss: 3.791351318359375
Epoch: 17, Loss: 3.780987024307251
Epoch: 18, Loss: 3.7712109088897705
Epoch: 19, Loss: 3.7618789672851562
Epoch: 20, Loss: 3.753053903579712
Epoch: 21, Loss: 3.7446439266204834
Epoch: 22, Loss: 3.7366271018981934
Epoch: 23, Loss: 3.7289466857910156
Epoch: 24, Loss: 3.721579074859619
Epoch: 25, Loss: 3.7145001888275146
Epoch: 26, Loss: 3.707683563232422
Epoch: 27, Loss: 3.701129674911499
Epoch: 28, Loss: 3.6

### Random walk, only infomax

In [71]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model = DeepGraphInfomax(hidden_channels=128,
                         encoder=Encoder(interm_channels=74, hidden_channels=128,
                                         num_features=128),
                         summary=lambda z, *args, **kwargs: torch.sigmoid(z.mean(dim=0)),
                         corruption=random_walk).to(device)
linkmodel = LinkPredHead()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005)
loss_bce = nn.BCEWithLogitsLoss()

In [72]:
num_epoch = 100
loss_epoch = []
for epoch in tqdm(range(1, num_epoch+1)):
  if epoch == num_epoch:
    pos_z, neg_z, summary, loss = train_only_infomax(epoch)
  else:
    loss = train_only_infomax(epoch)
  loss_epoch.append(loss)

  1%|          | 1/100 [00:03<05:57,  3.61s/it]

Epoch: 1, Loss: 1.711686372756958


  2%|▏         | 2/100 [00:07<06:35,  4.03s/it]

Epoch: 2, Loss: 1.4071383476257324


  3%|▎         | 3/100 [00:11<06:07,  3.79s/it]

...

Epoch: 98, Loss: 0.7245776653289795


 99%|█████████▉| 99/100 [06:12<00:03,  3.82s/it]

Epoch: 99, Loss: 1.1137895584106445


100%|██████████| 100/100 [06:15<00:00,  3.76s/it]

Epoch: 100, Loss: 0.4775034785270691


In [73]:
final_user_representations = {}
repr = pos_z
for i in range(pos_z.shape[0]):
  final_user_representations[i] = (i, repr[i].detach(), user_representations[i][1])

In [74]:
user_dataset = UserDataset(final_user_representations)

n = len(user_dataset)

indices = np.arange(n)
indices = np.random.permutation(indices)

train_indices = indices [:int(0.8*n)]
test_indices = indices[int(0.8*n):]

user_train_dataset = Subset(user_dataset, train_indices)
user_test_dataset = Subset(user_dataset, test_indices)

user_train_dataloader = DataLoader(user_train_dataset, batch_size=64, shuffle=True)
user_test_dataloader = DataLoader(user_test_dataset, batch_size=64, shuffle=True)

In [75]:
num_classes = 186
classes = np.arange(0, num_classes)

In [76]:
head_model = train2task()
torch.save(head_model.state_dict(), 'random_walk_only_infomax_model.pth')

Epoch: 0, Loss: 4.408990383148193
Epoch: 1, Loss: 4.213274955749512
Epoch: 2, Loss: 4.10835075378418
Epoch: 3, Loss: 4.036770820617676
Epoch: 4, Loss: 3.982470750808716
Epoch: 5, Loss: 3.9388766288757324
Epoch: 6, Loss: 3.9026286602020264
Epoch: 7, Loss: 3.8715105056762695
Epoch: 8, Loss: 3.8444066047668457
Epoch: 9, Loss: 3.820385456085205
Epoch: 10, Loss: 3.7988123893737793
Epoch: 11, Loss: 3.7792716026306152
Epoch: 12, Loss: 3.7614333629608154
Epoch: 13, Loss: 3.7449991703033447
Epoch: 14, Loss: 3.729807138442993
Epoch: 15, Loss: 3.7157046794891357
Epoch: 16, Loss: 3.7025179862976074
Epoch: 17, Loss: 3.690117120742798
Epoch: 18, Loss: 3.678478717803955
Epoch: 19, Loss: 3.6674821376800537
Epoch: 20, Loss: 3.6570746898651123
Epoch: 21, Loss: 3.6471683979034424
Epoch: 22, Loss: 3.637773036956787
Epoch: 23, Loss: 3.628807306289673
Epoch: 24, Loss: 3.620236873626709
Epoch: 25, Loss: 3.6120357513427734
Epoch: 26, Loss: 3.6041738986968994
Epoch: 27, Loss: 3.5966131687164307
Epoch: 28, Loss

### Node mix, Infomax + Link pred

In [77]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model = DeepGraphInfomax(hidden_channels=128,
                         encoder=Encoder(interm_channels=74, hidden_channels=128,
                                         num_features=128),
                         summary=lambda z, *args, **kwargs: torch.sigmoid(z.mean(dim=0)),
                         corruption=node_mix).to(device)
linkmodel = LinkPredHead()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005)
loss_bce = nn.BCEWithLogitsLoss()

In [78]:
num_epoch = 100
loss_epoch = []
for epoch in tqdm(range(1, num_epoch+1)):
  if epoch == num_epoch:
    pos_z, neg_z, summary, loss = train_infomax_linkpred(epoch)
  else:
    loss = train_infomax_linkpred(epoch)
  loss_epoch.append(loss)

  1%|          | 1/100 [00:09<15:38,  9.48s/it]

Epoch: 1, Loss: 3.34798264503479


  2%|▏         | 2/100 [00:16<13:18,  8.15s/it]

Epoch: 2, Loss: 2.848660707473755


  3%|▎         | 3/100 [00:23<12:09,  7.52s/it]

...

Epoch: 98, Loss: 0.7284792065620422


 99%|█████████▉| 99/100 [11:41<00:07,  7.10s/it]

Epoch: 99, Loss: 0.8222225904464722


100%|██████████| 100/100 [11:48<00:00,  7.09s/it]

Epoch: 100, Loss: 0.9420307874679565


In [79]:
final_user_representations = {}
repr = pos_z
for i in range(pos_z.shape[0]):
  final_user_representations[i] = (i, repr[i].detach(), user_representations[i][1])

In [80]:
user_dataset = UserDataset(final_user_representations)

n = len(user_dataset)

indices = np.arange(n)
indices = np.random.permutation(indices)

train_indices = indices [:int(0.8*n)]
test_indices = indices[int(0.8*n):]

user_train_dataset = Subset(user_dataset, train_indices)
user_test_dataset = Subset(user_dataset, test_indices)

user_train_dataloader = DataLoader(user_train_dataset, batch_size=64, shuffle=True)
user_test_dataloader = DataLoader(user_test_dataset, batch_size=64, shuffle=True)

In [81]:
num_classes = 186
classes = np.arange(0, num_classes)

In [82]:
head_model = train2task()
torch.save(head_model.state_dict(), 'node_mix_infomax_linkpred_model.pth')

Epoch: 0, Loss: 4.320979595184326
Epoch: 1, Loss: 4.219214916229248
Epoch: 2, Loss: 4.162557125091553
Epoch: 3, Loss: 4.1225810050964355
Epoch: 4, Loss: 4.091488838195801
Epoch: 5, Loss: 4.066256046295166
Epoch: 6, Loss: 4.044521331787109
Epoch: 7, Loss: 4.025753021240234
Epoch: 8, Loss: 4.0090861320495605
Epoch: 9, Loss: 3.9941675662994385
Epoch: 10, Loss: 3.980618953704834
Epoch: 11, Loss: 3.9680333137512207
Epoch: 12, Loss: 3.9565818309783936
Epoch: 13, Loss: 3.945878267288208
Epoch: 14, Loss: 3.9358906745910645
Epoch: 15, Loss: 3.9264256954193115
Epoch: 16, Loss: 3.917603015899658
Epoch: 17, Loss: 3.909235715866089
Epoch: 18, Loss: 3.901371717453003
Epoch: 19, Loss: 3.8938417434692383
Epoch: 20, Loss: 3.8866257667541504
Epoch: 21, Loss: 3.879770517349243
Epoch: 22, Loss: 3.873267412185669
Epoch: 23, Loss: 3.8669345378875732
Epoch: 24, Loss: 3.8609118461608887
Epoch: 25, Loss: 3.8550972938537598
Epoch: 26, Loss: 3.849501848220825
Epoch: 27, Loss: 3.844071626663208
Epoch: 28, Loss: 3

### Node dropout, Infomax + Link pred

In [83]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model = DeepGraphInfomax(hidden_channels=128,
                         encoder=Encoder(interm_channels=74, hidden_channels=128,
                                         num_features=128),
                         summary=lambda z, *args, **kwargs: torch.sigmoid(z.mean(dim=0)),
                         corruption=node_dropout).to(device)
linkmodel = LinkPredHead()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005)
loss_bce = nn.BCEWithLogitsLoss()

In [84]:
num_epoch = 100
loss_epoch = []
for epoch in tqdm(range(1, num_epoch+1)):
  if epoch == num_epoch:
    pos_z, neg_z, summary, loss = train_infomax_linkpred(epoch)
  else:
    loss = train_infomax_linkpred(epoch)
  loss_epoch.append(loss)

  1%|          | 1/100 [00:07<13:10,  7.99s/it]

Epoch: 1, Loss: 3.664515256881714


  2%|▏         | 2/100 [00:16<13:31,  8.28s/it]

Epoch: 2, Loss: 3.490450859069824


  3%|▎         | 3/100 [00:25<13:42,  8.47s/it]

...

Epoch: 98, Loss: 3.2625858783721924


 99%|█████████▉| 99/100 [13:45<00:08,  8.40s/it]

Epoch: 99, Loss: 3.368649482727051


100%|██████████| 100/100 [13:53<00:00,  8.34s/it]

Epoch: 100, Loss: 2.653125047683716


In [85]:
final_user_representations = {}
repr = pos_z
for i in range(pos_z.shape[0]):
  final_user_representations[i] = (i, repr[i].detach(), user_representations[i][1])

In [86]:
user_dataset = UserDataset(final_user_representations)

n = len(user_dataset)

indices = np.arange(n)
indices = np.random.permutation(indices)

train_indices = indices [:int(0.8*n)]
test_indices = indices[int(0.8*n):]

user_train_dataset = Subset(user_dataset, train_indices)
user_test_dataset = Subset(user_dataset, test_indices)

user_train_dataloader = DataLoader(user_train_dataset, batch_size=64, shuffle=True)
user_test_dataloader = DataLoader(user_test_dataset, batch_size=64, shuffle=True)

In [87]:
num_classes = 186
classes = np.arange(0, num_classes)

In [88]:
head_model = train2task()
torch.save(head_model.state_dict(), 'node_dropout_infomax_linkpred_model.pth')

Epoch: 0, Loss: 4.470142841339111
Epoch: 1, Loss: 4.250609874725342
Epoch: 2, Loss: 4.13272762298584
Epoch: 3, Loss: 4.051883697509766
Epoch: 4, Loss: 3.9903104305267334
Epoch: 5, Loss: 3.9406611919403076
Epoch: 6, Loss: 3.899014472961426
Epoch: 7, Loss: 3.863306999206543
Epoch: 8, Loss: 3.832075595855713
Epoch: 9, Loss: 3.804293632507324
Epoch: 10, Loss: 3.7793848514556885
Epoch: 11, Loss: 3.756803274154663
Epoch: 12, Loss: 3.736203908920288
Epoch: 13, Loss: 3.7172863483428955
Epoch: 14, Loss: 3.699819326400757
Epoch: 15, Loss: 3.6835925579071045
Epoch: 16, Loss: 3.6684701442718506
Epoch: 17, Loss: 3.6543116569519043
Epoch: 18, Loss: 3.6410152912139893
Epoch: 19, Loss: 3.6284961700439453
Epoch: 20, Loss: 3.6166610717773438
Epoch: 21, Loss: 3.605457067489624
Epoch: 22, Loss: 3.5947988033294678
Epoch: 23, Loss: 3.5846564769744873
Epoch: 24, Loss: 3.5749940872192383
Epoch: 25, Loss: 3.565760850906372
Epoch: 26, Loss: 3.5569303035736084
Epoch: 27, Loss: 3.548447608947754
Epoch: 28, Loss: 

### Edge dropout, Infomax + Link pred

In [89]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model = DeepGraphInfomax(hidden_channels=128,
                         encoder=Encoder(interm_channels=74, hidden_channels=128,
                                         num_features=128),
                         summary=lambda z, *args, **kwargs: torch.sigmoid(z.mean(dim=0)),
                         corruption=edge_dropout).to(device)
linkmodel = LinkPredHead()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005)
loss_bce = nn.BCEWithLogitsLoss()

In [90]:
num_epoch = 100
loss_epoch = []
for epoch in tqdm(range(1, num_epoch+1)):
  if epoch == num_epoch:
    pos_z, neg_z, summary, loss = train_infomax_linkpred(epoch)
  else:
    loss = train_infomax_linkpred(epoch)
  loss_epoch.append(loss)

  1%|          | 1/100 [00:06<10:03,  6.10s/it]

Epoch: 1, Loss: 3.43694806098938


  2%|▏         | 2/100 [00:12<10:36,  6.49s/it]

Epoch: 2, Loss: 3.374835968017578


  3%|▎         | 3/100 [00:18<10:03,  6.23s/it]

...

Epoch: 98, Loss: 1.4260797500610352


 99%|█████████▉| 99/100 [10:30<00:06,  6.22s/it]

Epoch: 99, Loss: 1.8688944578170776


100%|██████████| 100/100 [10:37<00:00,  6.37s/it]

Epoch: 100, Loss: 1.8372821807861328


In [91]:
final_user_representations = {}
repr = pos_z
for i in range(pos_z.shape[0]):
  final_user_representations[i] = (i, repr[i].detach(), user_representations[i][1])

In [92]:
user_dataset = UserDataset(final_user_representations)

n = len(user_dataset)

indices = np.arange(n)
indices = np.random.permutation(indices)

train_indices = indices [:int(0.8*n)]
test_indices = indices[int(0.8*n):]

user_train_dataset = Subset(user_dataset, train_indices)
user_test_dataset = Subset(user_dataset, test_indices)

user_train_dataloader = DataLoader(user_train_dataset, batch_size=64, shuffle=True)
user_test_dataloader = DataLoader(user_test_dataset, batch_size=64, shuffle=True)

In [93]:
num_classes = 186
classes = np.arange(0, num_classes)

In [94]:
head_model = train2task()
torch.save(head_model.state_dict(), 'edge_dropout_infomax_linkpred_model.pth')

Epoch: 0, Loss: 4.329938888549805
Epoch: 1, Loss: 4.177267074584961
Epoch: 2, Loss: 4.094830513000488
Epoch: 3, Loss: 4.038265705108643
Epoch: 4, Loss: 3.994664430618286
Epoch: 5, Loss: 3.959409236907959
Epoch: 6, Loss: 3.9297213554382324
Epoch: 7, Loss: 3.903956890106201
Epoch: 8, Loss: 3.881286859512329
Epoch: 9, Loss: 3.8610832691192627
Epoch: 10, Loss: 3.8427982330322266
Epoch: 11, Loss: 3.8262059688568115
Epoch: 12, Loss: 3.8109612464904785
Epoch: 13, Loss: 3.796868324279785
Epoch: 14, Loss: 3.783818483352661
Epoch: 15, Loss: 3.771604061126709
Epoch: 16, Loss: 3.760194778442383
Epoch: 17, Loss: 3.7494356632232666
Epoch: 18, Loss: 3.739291191101074
Epoch: 19, Loss: 3.7297346591949463
Epoch: 20, Loss: 3.7206382751464844
Epoch: 21, Loss: 3.712029218673706
Epoch: 22, Loss: 3.703831195831299
Epoch: 23, Loss: 3.6959879398345947
Epoch: 24, Loss: 3.6885006427764893
Epoch: 25, Loss: 3.6813390254974365
Epoch: 26, Loss: 3.6744658946990967
Epoch: 27, Loss: 3.667891263961792
Epoch: 28, Loss: 3

### Random walk, Infomax + Link pred

In [95]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model = DeepGraphInfomax(hidden_channels=128,
                         encoder=Encoder(interm_channels=74, hidden_channels=128,
                                         num_features=128),
                         summary=lambda z, *args, **kwargs: torch.sigmoid(z.mean(dim=0)),
                         corruption=random_walk).to(device)
linkmodel = LinkPredHead()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005)
loss_bce = nn.BCEWithLogitsLoss()

In [96]:
num_epoch = 100
loss_epoch = []
for epoch in tqdm(range(1, num_epoch+1)):
  if epoch == num_epoch:
    pos_z, neg_z, summary, loss = train_infomax_linkpred(epoch)
  else:
    loss = train_infomax_linkpred(epoch)
  loss_epoch.append(loss)

  1%|          | 1/100 [00:03<06:35,  4.00s/it]

Epoch: 1, Loss: 4.045363903045654


  2%|▏         | 2/100 [00:08<07:14,  4.43s/it]

Epoch: 2, Loss: 3.7003698348999023


  3%|▎         | 3/100 [00:12<06:45,  4.18s/it]

...

Epoch: 98, Loss: 1.3365979194641113


 99%|█████████▉| 99/100 [06:53<00:04,  4.13s/it]

Epoch: 99, Loss: 1.445319652557373


100%|██████████| 100/100 [06:57<00:00,  4.18s/it]

Epoch: 100, Loss: 1.634057641029358


In [97]:
final_user_representations = {}
repr = pos_z
for i in range(pos_z.shape[0]):
  final_user_representations[i] = (i, repr[i].detach(), user_representations[i][1])

In [98]:
user_dataset = UserDataset(final_user_representations)

n = len(user_dataset)

indices = np.arange(n)
indices = np.random.permutation(indices)

train_indices = indices [:int(0.8*n)]
test_indices = indices[int(0.8*n):]

user_train_dataset = Subset(user_dataset, train_indices)
user_test_dataset = Subset(user_dataset, test_indices)

user_train_dataloader = DataLoader(user_train_dataset, batch_size=64, shuffle=True)
user_test_dataloader = DataLoader(user_test_dataset, batch_size=64, shuffle=True)

In [99]:
num_classes = 186
classes = np.arange(0, num_classes)

In [100]:
head_model = train2task()
torch.save(head_model.state_dict(), 'random_walk_infomax_linkpred_model.pth')

Epoch: 0, Loss: 4.424149513244629
Epoch: 1, Loss: 4.248197078704834
Epoch: 2, Loss: 4.152710914611816
Epoch: 3, Loss: 4.08593225479126
Epoch: 4, Loss: 4.034071922302246
Epoch: 5, Loss: 3.991478204727173
Epoch: 6, Loss: 3.9552371501922607
Epoch: 7, Loss: 3.923595666885376
Epoch: 8, Loss: 3.895565986633301
Epoch: 9, Loss: 3.8702945709228516
Epoch: 10, Loss: 3.847313165664673
Epoch: 11, Loss: 3.8262948989868164
Epoch: 12, Loss: 3.806889057159424
Epoch: 13, Loss: 3.7889153957366943
Epoch: 14, Loss: 3.772141456604004
Epoch: 15, Loss: 3.7564241886138916
Epoch: 16, Loss: 3.741687297821045
Epoch: 17, Loss: 3.7277867794036865
Epoch: 18, Loss: 3.714641809463501
Epoch: 19, Loss: 3.7021484375
Epoch: 20, Loss: 3.690312385559082
Epoch: 21, Loss: 3.679049015045166
Epoch: 22, Loss: 3.668288469314575
Epoch: 23, Loss: 3.6580002307891846
Epoch: 24, Loss: 3.6481425762176514
Epoch: 25, Loss: 3.638688087463379
Epoch: 26, Loss: 3.629626750946045
Epoch: 27, Loss: 3.6208925247192383
Epoch: 28, Loss: 3.61247682